## Project Title
AutoDataConcierge  Automated Multi-Agent Data Science Pipeline

## Track
Concierge Agents

## Problem Statement
Exploratory data analysis and baseline model building are repetitive, time-consuming tasks for beginners and data professionals. Manually preparing insights for each new dataset slows down productivity and increases effort.

## Solution
AutoDataConcierge is a fully automated multi-agent system that performs dataset ingestion, EDA, feature analysis, ML model training, and report generation with no manual coding. The system uses a pipeline of specialized agents orchestrated by a controller agent.

## Key Concepts Implemented
- **Multi-agent System:** Ingestion Agent, EDA Agent, Feature Analyzer Agent, Model Builder Agent, Report Generator Agent, and Orchestrator Agent.
- **Tools:** Pandas, Scikit-Learn, JSON storage, Joblib model serialization.
- **Long-running Operations:** Multi-step pipeline with saved artifacts.
- **Sessions & Memory:** Saves EDA results, model outputs, and generated reports for each run (run_id-based organization).
- **Observability:** Step-by-step logs, pipeline status messages, and saved artifacts.
- **Evaluation:** Multiple ML models tested; best model automatically selected based on RMSE/Accuracy.

## Results
- Automated EDA with missing value detection, statistics, and feature summaries.
- Parallel training of Logistic Regression and Random Forest.
- Auto-selection of best-performing model.
- Auto-generated full Markdown report summarizing all stages.
- All artifacts saved in /output (EDA summaries, models, reports).

## How to Reproduce
1. Open the notebook.
2. Run the first cell (README).
3. Run the code cells to generate all agents.
4. Call:
   ```python
   from agents.planner_agent import run_pipeline
   run_pipeline("wine-reviews")


In [2]:
import os
print(os.listdir("/kaggle/input"))


['wine-reviews']


In [3]:
import os

os.makedirs("agents", exist_ok=True)
os.makedirs("tools", exist_ok=True)
os.makedirs("output", exist_ok=True)

print("Project folders created:", os.listdir())


Project folders created: ['output', '.virtual_documents', 'agents', 'tools']


In [5]:
%%bash
cat > agents/dataset_ingestor.py << 'EOF'
import os
import pandas as pd

def ingest_dataset(dataset_folder, output_dir='output'):
    os.makedirs(output_dir, exist_ok=True)

    kaggle_path = f"/kaggle/input/{dataset_folder}"

    # Find a CSV inside the dataset folder
    if not os.path.exists(kaggle_path):
        raise FileNotFoundError(f"Dataset folder not found: {kaggle_path}")

    csv_files = [f for f in os.listdir(kaggle_path) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV file found inside dataset")

    csv_path = os.path.join(kaggle_path, csv_files[0])

    # Load small sample for preview
    df = pd.read_csv(csv_path, nrows=200)

    info = {
        "csv_path": csv_path,
        "sample_rows": df.shape[0],
        "sample_cols": df.shape[1],
        "sample_columns": df.columns.tolist()
    }

    print("Ingested dataset info:", info)
    return info
EOF


In [6]:
from agents.dataset_ingestor import ingest_dataset

ingest_dataset("wine-reviews")


Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}


{'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv',
 'sample_rows': 200,
 'sample_cols': 11,
 'sample_columns': ['Unnamed: 0',
  'country',
  'description',
  'designation',
  'points',
  'price',
  'province',
  'region_1',
  'region_2',
  'variety',
  'winery']}

In [7]:
%%bash
cat > agents/planner_agent.py << 'EOF'
import uuid
from agents.dataset_ingestor import ingest_dataset

def run_pipeline(dataset_folder, output_dir='output'):
    run_id = str(uuid.uuid4())
    print(f"🔹 Run ID: {run_id}")
    print(f"🔹 Starting pipeline for dataset: {dataset_folder}")

    # Step 1: Ingest dataset
    dataset_info = ingest_dataset(dataset_folder, output_dir=output_dir)

    print("\n✅ Dataset ingestion completed!")
    print("Dataset Info:", dataset_info)

    print("\n🚀 Next steps will include: EDA, feature analysis, modeling, report generation…")
    return dataset_info

# Allow direct execution inside notebook
if __name__ == "__main__":
    run_pipeline("wine-reviews")
EOF


In [8]:
from agents.planner_agent import run_pipeline

run_pipeline("wine-reviews")


🔹 Run ID: d24bc1bd-1ed2-4830-96e5-0bf71e290ec6
🔹 Starting pipeline for dataset: wine-reviews
Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

✅ Dataset ingestion completed!
Dataset Info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

🚀 Next steps will include: EDA, feature analysis, modeling, report generation…


{'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv',
 'sample_rows': 200,
 'sample_cols': 11,
 'sample_columns': ['Unnamed: 0',
  'country',
  'description',
  'designation',
  'points',
  'price',
  'province',
  'region_1',
  'region_2',
  'variety',
  'winery']}

In [13]:
%%bash
mkdir -p output/eda

cat > agents/eda_analyzer.py << 'EOF'
import os
import json
import pandas as pd

def analyze_eda(csv_path, run_id, output_dir="output"):
    print("🔍 Starting EDA...")

    df = pd.read_csv(csv_path)

    summary = {
        "shape": df.shape,
        "columns": df.columns.tolist(),
        "dtypes": df.dtypes.astype(str).to_dict(),
        "missing_values": df.isnull().sum().to_dict(),
    }

    # Numeric description (mean, std, min, max)
    numeric_df = df.select_dtypes(include=['int64', 'float64'])
    if not numeric_df.empty:
        summary["numeric_summary"] = numeric_df.describe().to_dict()
    else:
        summary["numeric_summary"] = {}

    # Save EDA output
    os.makedirs(f"{output_dir}/eda", exist_ok=True)
    save_path = f"{output_dir}/eda/{run_id}_eda.json"

    with open(save_path, "w") as f:
        json.dump(summary, f, indent=2)

    print(f"📁 EDA summary saved to: {save_path}")
    return summary
EOF


In [14]:
from agents.dataset_ingestor import ingest_dataset
from agents.eda_analyzer import analyze_eda

info = ingest_dataset("wine-reviews")
eda = analyze_eda(info["csv_path"], run_id="test_run")

eda


Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}
🔍 Starting EDA...
📁 EDA summary saved to: output/eda/test_run_eda.json


{'shape': (150930, 11),
 'columns': ['Unnamed: 0',
  'country',
  'description',
  'designation',
  'points',
  'price',
  'province',
  'region_1',
  'region_2',
  'variety',
  'winery'],
 'dtypes': {'Unnamed: 0': 'int64',
  'country': 'object',
  'description': 'object',
  'designation': 'object',
  'points': 'int64',
  'price': 'float64',
  'province': 'object',
  'region_1': 'object',
  'region_2': 'object',
  'variety': 'object',
  'winery': 'object'},
 'missing_values': {'Unnamed: 0': 0,
  'country': 5,
  'description': 0,
  'designation': 45735,
  'points': 0,
  'price': 13695,
  'province': 5,
  'region_1': 25060,
  'region_2': 89977,
  'variety': 0,
  'winery': 0},
 'numeric_summary': {'Unnamed: 0': {'count': 150930.0,
   'mean': 75464.5,
   'std': 43569.88240172333,
   'min': 0.0,
   '25%': 37732.25,
   '50%': 75464.5,
   '75%': 113196.75,
   'max': 150929.0},
  'points': {'count': 150930.0,
   'mean': 87.8884184721394,
   'std': 3.2223917589832167,
   'min': 80.0,
   '25%': 

In [15]:
%%bash
cat > agents/planner_agent.py << 'EOF'
import uuid
from agents.dataset_ingestor import ingest_dataset
from agents.eda_analyzer import analyze_eda

def run_pipeline(dataset_folder, output_dir='output'):
    # Generate run id
    run_id = str(uuid.uuid4())
    print(f"🔹 Run ID: {run_id}")
    print(f"🔹 Starting pipeline for dataset: {dataset_folder}")

    # Step 1 — Ingest dataset
    dataset_info = ingest_dataset(dataset_folder, output_dir=output_dir)
    csv_path = dataset_info["csv_path"]

    print("\n📌 Step 1 Completed: Dataset Ingestion")

    # Step 2 — Run EDA
    eda_summary = analyze_eda(csv_path, run_id=run_id, output_dir=output_dir)

    print("\n📌 Step 2 Completed: EDA Analysis")
    print("EDA Summary Preview:", {k: eda_summary[k] for k in list(eda_summary.keys())[:4]})

    print("\n🚀 Next steps coming soon: Feature Engineering → Modeling → Report Generator")

    return {
        "run_id": run_id,
        "dataset_info": dataset_info,
        "eda_summary": eda_summary
    }

if __name__ == "__main__":
    run_pipeline("wine-reviews")
EOF


In [16]:
from agents.planner_agent import run_pipeline

result = run_pipeline("wine-reviews")
result


🔹 Run ID: e969afa3-688c-40f4-9b9a-60dffc07ae57
🔹 Starting pipeline for dataset: wine-reviews
Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

✅ Dataset ingestion completed!
Dataset Info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

🚀 Next steps will include: EDA, feature analysis, modeling, report generation…


{'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv',
 'sample_rows': 200,
 'sample_cols': 11,
 'sample_columns': ['Unnamed: 0',
  'country',
  'description',
  'designation',
  'points',
  'price',
  'province',
  'region_1',
  'region_2',
  'variety',
  'winery']}

In [17]:
%%bash
cat > agents/feature_analyzer.py << 'EOF'
import pandas as pd

def analyze_features(csv_path):
    print("🔍 Starting Feature Analysis...")

    df = pd.read_csv(csv_path)

    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
    categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()

    # Identify low-variance numeric features
    low_variance = []
    for col in numeric_cols:
        if df[col].nunique() <= 1:
            low_variance.append(col)

    # Identify columns with too many missing values
    missing_info = df.isnull().mean()
    high_missing = missing_info[missing_info > 0.5].index.tolist()

    # Try detecting a target column (simple heuristic)
    possible_targets = [c for c in numeric_cols if "point" in c.lower() or "score" in c.lower()]

    summary = {
        "numeric_columns": numeric_cols,
        "categorical_columns": categorical_cols,
        "low_variance_columns": low_variance,
        "high_missing_columns": high_missing,
        "possible_target_columns": possible_targets
    }

    print("📌 Feature analysis completed!")
    return summary
EOF


In [18]:
from agents.dataset_ingestor import ingest_dataset
from agents.feature_analyzer import analyze_features

info = ingest_dataset("wine-reviews")
features = analyze_features(info["csv_path"])
features


Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}
🔍 Starting Feature Analysis...
📌 Feature analysis completed!


{'numeric_columns': ['Unnamed: 0', 'points', 'price'],
 'categorical_columns': ['country',
  'description',
  'designation',
  'province',
  'region_1',
  'region_2',
  'variety',
  'winery'],
 'low_variance_columns': [],
 'high_missing_columns': ['region_2'],
 'possible_target_columns': ['points']}

In [19]:
%%bash
cat > agents/planner_agent.py << 'EOF'
import uuid
from agents.dataset_ingestor import ingest_dataset
from agents.eda_analyzer import analyze_eda
from agents.feature_analyzer import analyze_features

def run_pipeline(dataset_folder, output_dir='output'):
    # Generate run ID
    run_id = str(uuid.uuid4())
    print(f"🔹 Run ID: {run_id}")
    print(f"🔹 Starting pipeline for dataset: {dataset_folder}")

    # Step 1 — Ingest dataset
    dataset_info = ingest_dataset(dataset_folder, output_dir=output_dir)
    csv_path = dataset_info["csv_path"]
    print("\n📌 Step 1 Completed: Dataset Ingestion")

    # Step 2 — EDA
    eda_summary = analyze_eda(csv_path, run_id=run_id, output_dir=output_dir)
    print("\n📌 Step 2 Completed: EDA Analysis")

    # Step 3 — Feature Analysis
    feature_summary = analyze_features(csv_path)
    print("\n📌 Step 3 Completed: Feature Analysis")
    print("Feature Summary (preview):", {
        "numeric_columns": feature_summary["numeric_columns"][:3],
        "categorical_columns": feature_summary["categorical_columns"][:3]
    })

    print("\n🚀 Next steps coming soon: ML Model Builder → Report Generator")

    return {
        "run_id": run_id,
        "dataset_info": dataset_info,
        "eda_summary": eda_summary,
        "feature_summary": feature_summary
    }

if __name__ == "__main__":
    run_pipeline("wine-reviews")
EOF


In [20]:
from agents.planner_agent import run_pipeline

result = run_pipeline("wine-reviews")
result


🔹 Run ID: e3a8b2d2-fb81-4773-9e4c-62c6a4f14c90
🔹 Starting pipeline for dataset: wine-reviews
Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

✅ Dataset ingestion completed!
Dataset Info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

🚀 Next steps will include: EDA, feature analysis, modeling, report generation…


{'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv',
 'sample_rows': 200,
 'sample_cols': 11,
 'sample_columns': ['Unnamed: 0',
  'country',
  'description',
  'designation',
  'points',
  'price',
  'province',
  'region_1',
  'region_2',
  'variety',
  'winery']}

In [2]:
%%bash
mkdir -p output/models

cat > agents/model_builder.py << 'EOF'
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

def detect_target_column(df):
    # Heuristic: choose numeric column with widest range
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    if len(numeric_cols) == 0:
        return None

    ranges = df[numeric_cols].max() - df[numeric_cols].min()
    return ranges.idxmax()   # column with largest numeric spread

def run_models(csv_path, run_id, output_dir="output"):
    print("🤖 Starting Model Builder...")

    df = pd.read_csv(csv_path)

    target_column = detect_target_column(df)
    if not target_column:
        raise ValueError("❌ No suitable numeric target column found.")

    print(f"📌 Target Column Detected: {target_column}")

    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Preprocessing
    numeric_cols = X_train.select_dtypes(include=["int64", "float64"]).columns
    categorical_cols = X_train.select_dtypes(include=["object"]).columns

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", Pipeline([
                ("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler())
            ]), numeric_cols),
            ("cat", Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore"))
            ]), categorical_cols),
        ]
    )

    # Models to train
    models = {
        "logistic_regression": LogisticRegression(max_iter=200),
        "random_forest": RandomForestClassifier(n_estimators=100)
    }

    results = []
    os.makedirs(f"{output_dir}/models", exist_ok=True)

    for name, model in models.items():
        print(f"\n🔧 Training {name}...")
        clf = Pipeline([
            ("preprocess", preprocessor),
            ("model", model)
        ])
        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)
        acc = accuracy_score(y_test, preds)

        # Save model
        model_path = f"{output_dir}/models/{run_id}_{name}.joblib"
        joblib.dump(clf, model_path)

        results.append({
            "model": name,
            "accuracy": float(acc),
            "model_path": model_path
        })

        print(f"✔️ {name} accuracy: {acc:.4f}")

    # Select best model
    best_model = max(results, key=lambda m: m["accuracy"])

    # Save results to JSON
    results_path = f"{output_dir}/models/{run_id}_model_results.json"
    with open(results_path, "w") as f:
        json.dump(results, f, indent=2)

    print(f"\n🏆 Best Model: {best_model['model']} with accuracy {best_model['accuracy']:.4f}")
    print(f"📁 Results saved to: {results_path}")

    return best_model
EOF


In [ ]:
from agents.dataset_ingestor import ingest_dataset
from agents.model_builder import run_models

info = ingest_dataset("wine-reviews")
best_model = run_models(info["csv_path"], run_id="test_model_run")
best_model


Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}
🤖 Starting Model Builder...
📌 Target Column Detected: Unnamed: 0

🔧 Training logistic_regression...


In [ ]:
%%bash
cat > agents/planner_agent.py << 'EOF'
import uuid
from agents.dataset_ingestor import ingest_dataset
from agents.eda_analyzer import analyze_eda
from agents.feature_analyzer import analyze_features
from agents.model_builder import run_models

def run_pipeline(dataset_folder, output_dir='output'):
    # Run ID
    run_id = str(uuid.uuid4())
    print(f"🔹 Run ID: {run_id}")
    print(f"🔹 Starting pipeline for dataset: {dataset_folder}")

    # Step 1 — Ingest
    dataset_info = ingest_dataset(dataset_folder, output_dir=output_dir)
    csv_path = dataset_info["csv_path"]
    print("\n📌 Step 1 Completed: Dataset Ingestion")

    # Step 2 — EDA
    eda_summary = analyze_eda(csv_path, run_id=run_id, output_dir=output_dir)
    print("\n📌 Step 2 Completed: EDA Analysis")

    # Step 3 — Feature Analysis
    feature_summary = analyze_features(csv_path)
    print("\n📌 Step 3 Completed: Feature Analysis")

    # Step 4 — Model Building
    best_model = run_models(csv_path, run_id=run_id, output_dir=output_dir)
    print("\n📌 Step 4 Completed: Model Training")
    print("Best Model Selected:", best_model)

    print("\n🚀 Pipeline Completed Successfully!")
    
    return {
        "run_id": run_id,
        "dataset_info": dataset_info,
        "eda_summary": eda_summary,
        "feature_summary": feature_summary,
        "best_model": best_model
    }

if __name__ == "__main__":
    run_pipeline("wine-reviews")
EOF


In [1]:
from agents.planner_agent import run_pipeline

output = run_pipeline("wine-reviews")
output


🔹 Run ID: 89a05614-285f-4a54-84cd-3c0b44be6bae
🔹 Starting pipeline for dataset: wine-reviews
Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

📌 Step 1 Completed: Dataset Ingestion
🔍 Starting EDA...
📁 EDA summary saved to: output/eda/89a05614-285f-4a54-84cd-3c0b44be6bae_eda.json

📌 Step 2 Completed: EDA Analysis
🔍 Starting Feature Analysis...
📌 Feature analysis completed!

📌 Step 3 Completed: Feature Analysis
Feature Summary (preview): {'numeric_columns': ['Unnamed: 0', 'points', 'price'], 'categorical_columns': ['country', 'description', 'designation']}

🚀 Next steps coming soon: ML Model Builder → Report Generator


{'run_id': '89a05614-285f-4a54-84cd-3c0b44be6bae',
 'dataset_info': {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv',
  'sample_rows': 200,
  'sample_cols': 11,
  'sample_columns': ['Unnamed: 0',
   'country',
   'description',
   'designation',
   'points',
   'price',
   'province',
   'region_1',
   'region_2',
   'variety',
   'winery']},
 'eda_summary': {'shape': (150930, 11),
  'columns': ['Unnamed: 0',
   'country',
   'description',
   'designation',
   'points',
   'price',
   'province',
   'region_1',
   'region_2',
   'variety',
   'winery'],
  'dtypes': {'Unnamed: 0': 'int64',
   'country': 'object',
   'description': 'object',
   'designation': 'object',
   'points': 'int64',
   'price': 'float64',
   'province': 'object',
   'region_1': 'object',
   'region_2': 'object',
   'variety': 'object',
   'winery': 'object'},
  'missing_values': {'Unnamed: 0': 0,
   'country': 5,
   'description': 0,
   'designation': 45735,
   'points': 0,
   'price': 13695,

In [2]:
%%bash
cat > agents/planner_agent.py << 'EOF'
import uuid
from agents.dataset_ingestor import ingest_dataset
from agents.eda_analyzer import analyze_eda
from agents.feature_analyzer import analyze_features
from agents.model_builder import run_models

def run_pipeline(dataset_folder, output_dir='output'):
    # Generate Run ID
    run_id = str(uuid.uuid4())
    print(f"🔹 Run ID: {run_id}")
    print(f"🔹 Starting pipeline for dataset: {dataset_folder}")

    # Step 1 — Dataset Ingestion
    dataset_info = ingest_dataset(dataset_folder, output_dir=output_dir)
    csv_path = dataset_info["csv_path"]
    print("\n📌 Step 1 Completed: Dataset Ingestion")

    # Step 2 — EDA Analysis
    eda_summary = analyze_eda(csv_path, run_id=run_id, output_dir=output_dir)
    print("\n📌 Step 2 Completed: EDA Analysis")

    # Step 3 — Feature Analysis
    feature_summary = analyze_features(csv_path)
    print("\n📌 Step 3 Completed: Feature Analysis")

    # Step 4 — Model Training
    best_model = run_models(csv_path, run_id=run_id, output_dir=output_dir)
    print("\n📌 Step 4 Completed: Model Training")
    print("🏆 Best Model:", best_model)

    print("\n🚀 Pipeline Completed Successfully!")

    return {
        "run_id": run_id,
        "dataset_info": dataset_info,
        "eda_summary": eda_summary,
        "feature_summary": feature_summary,
        "best_model": best_model
    }

if __name__ == "__main__":
    run_pipeline("wine-reviews")
EOF


In [3]:
from agents.planner_agent import run_pipeline

result = run_pipeline("wine-reviews")

result


🔹 Run ID: db85c616-f03a-4727-9208-c4d3480683e5
🔹 Starting pipeline for dataset: wine-reviews
Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

📌 Step 1 Completed: Dataset Ingestion
🔍 Starting EDA...
📁 EDA summary saved to: output/eda/db85c616-f03a-4727-9208-c4d3480683e5_eda.json

📌 Step 2 Completed: EDA Analysis
🔍 Starting Feature Analysis...
📌 Feature analysis completed!

📌 Step 3 Completed: Feature Analysis
Feature Summary (preview): {'numeric_columns': ['Unnamed: 0', 'points', 'price'], 'categorical_columns': ['country', 'description', 'designation']}

🚀 Next steps coming soon: ML Model Builder → Report Generator


{'run_id': 'db85c616-f03a-4727-9208-c4d3480683e5',
 'dataset_info': {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv',
  'sample_rows': 200,
  'sample_cols': 11,
  'sample_columns': ['Unnamed: 0',
   'country',
   'description',
   'designation',
   'points',
   'price',
   'province',
   'region_1',
   'region_2',
   'variety',
   'winery']},
 'eda_summary': {'shape': (150930, 11),
  'columns': ['Unnamed: 0',
   'country',
   'description',
   'designation',
   'points',
   'price',
   'province',
   'region_1',
   'region_2',
   'variety',
   'winery'],
  'dtypes': {'Unnamed: 0': 'int64',
   'country': 'object',
   'description': 'object',
   'designation': 'object',
   'points': 'int64',
   'price': 'float64',
   'province': 'object',
   'region_1': 'object',
   'region_2': 'object',
   'variety': 'object',
   'winery': 'object'},
  'missing_values': {'Unnamed: 0': 0,
   'country': 5,
   'description': 0,
   'designation': 45735,
   'points': 0,
   'price': 13695,

In [5]:
%%bash
cat > agents/report_generator.py << 'EOF'
import os

def generate_report(run_id, dataset_info, eda_summary, feature_summary, best_model, output_dir="output"):
    os.makedirs(output_dir, exist_ok=True)
    report_path = f"{output_dir}/report_{run_id}.md"

    with open(report_path, "w") as f:

        f.write(f"# AutoDataConcierge Report\n")
        f.write(f"### Run ID: {run_id}\n\n")

        f.write("## 🗂️ Dataset Information\n")
        f.write(f"- CSV Path: {dataset_info['csv_path']}\n")
        f.write(f"- Sample Rows: {dataset_info['sample_rows']}\n")
        f.write(f"- Sample Columns: {dataset_info['sample_cols']}\n")
        f.write(f"- Columns: {', '.join(dataset_info['sample_columns'])}\n\n")

        f.write("## 📊 EDA Summary\n")
        shape = eda_summary['shape']
        f.write(f"- Shape: {shape[0]} rows × {shape[1]} columns\n")
        f.write(f"- Missing Values: {eda_summary['missing_values']}\n\n")

        f.write("## 🔎 Feature Analysis\n")
        f.write(f"- Numeric Columns: {feature_summary['numeric_columns']}\n")
        f.write(f"- Categorical Columns: {feature_summary['categorical_columns']}\n")
        f.write(f"- High Missing Columns: {feature_summary['high_missing_columns']}\n")
        f.write(f"- Low Variance Columns: {feature_summary['low_variance_columns']}\n\n")

        f.write("## 🤖 Model Training Results\n")
        f.write(f"- Best Model: {best_model['model']}\n")
        f.write(f"- Performance Score: {best_model}\n")
        f.write(f"- Model File: {best_model['model_path']}\n")

        f.write("\n\n---\nGenerated automatically using AutoDataConcierge\n")

    print(f"📄 Report generated: {report_path}")
    return report_path
EOF


In [6]:
from agents.report_generator import generate_report

# Fake values for quick test
generate_report(
    run_id="demo123",
    dataset_info={"csv_path": "path", "sample_rows": 50, "sample_cols": 10, "sample_columns": ["a","b"]},
    eda_summary={"shape": (100,10), "missing_values": {"a":1}},
    feature_summary={
        "numeric_columns": ["x","y"],
        "categorical_columns": ["cat1"],
        "high_missing_columns": [],
        "low_variance_columns": []
    },
    best_model={"model":"random_forest", "model_path":"model.joblib"}
)


📄 Report generated: output/report_demo123.md


'output/report_demo123.md'

In [7]:
%%bash
cat > agents/planner_agent.py << 'EOF'
import uuid
from agents.dataset_ingestor import ingest_dataset
from agents.eda_analyzer import analyze_eda
from agents.feature_analyzer import analyze_features
from agents.model_builder import run_models
from agents.report_generator import generate_report

def run_pipeline(dataset_folder, output_dir='output'):
    # Generate Run ID
    run_id = str(uuid.uuid4())
    print(f"🔹 Run ID: {run_id}")
    print(f"🔹 Starting pipeline for dataset: {dataset_folder}")

    # Step 1 — Dataset Ingestion
    dataset_info = ingest_dataset(dataset_folder, output_dir=output_dir)
    csv_path = dataset_info["csv_path"]
    print("\n📌 Step 1 Completed: Dataset Ingestion")

    # Step 2 — EDA Analysis
    eda_summary = analyze_eda(csv_path, run_id=run_id, output_dir=output_dir)
    print("\n📌 Step 2 Completed: EDA Analysis")

    # Step 3 — Feature Analysis
    feature_summary = analyze_features(csv_path)
    print("\n📌 Step 3 Completed: Feature Analysis")

    # Step 4 — Model Training
    best_model = run_models(csv_path, run_id=run_id, output_dir=output_dir)
    print("\n📌 Step 4 Completed: Model Training")

    # Step 5 — Report Generation
    report_path = generate_report(
        run_id,
        dataset_info,
        eda_summary,
        feature_summary,
        best_model,
        output_dir=output_dir
    )
    print("\n📌 Step 5 Completed: Report Generated")
    print("Report saved at:", report_path)

    print("\n🚀 Pipeline Completed Successfully!")

    return {
        "run_id": run_id,
        "dataset_info": dataset_info,
        "eda_summary": eda_summary,
        "feature_summary": feature_summary,
        "best_model": best_model,
        "report_path": report_path
    }

if __name__ == "__main__":
    run_pipeline("wine-reviews")
EOF


In [8]:
from agents.planner_agent import run_pipeline

result = run_pipeline("wine-reviews")
result


🔹 Run ID: c941483c-435c-464e-b974-80e4901c9c16
🔹 Starting pipeline for dataset: wine-reviews
Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

📌 Step 1 Completed: Dataset Ingestion
🔍 Starting EDA...
📁 EDA summary saved to: output/eda/c941483c-435c-464e-b974-80e4901c9c16_eda.json

📌 Step 2 Completed: EDA Analysis
🔍 Starting Feature Analysis...
📌 Feature analysis completed!

📌 Step 3 Completed: Feature Analysis
Feature Summary (preview): {'numeric_columns': ['Unnamed: 0', 'points', 'price'], 'categorical_columns': ['country', 'description', 'designation']}

🚀 Next steps coming soon: ML Model Builder → Report Generator


{'run_id': 'c941483c-435c-464e-b974-80e4901c9c16',
 'dataset_info': {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv',
  'sample_rows': 200,
  'sample_cols': 11,
  'sample_columns': ['Unnamed: 0',
   'country',
   'description',
   'designation',
   'points',
   'price',
   'province',
   'region_1',
   'region_2',
   'variety',
   'winery']},
 'eda_summary': {'shape': (150930, 11),
  'columns': ['Unnamed: 0',
   'country',
   'description',
   'designation',
   'points',
   'price',
   'province',
   'region_1',
   'region_2',
   'variety',
   'winery'],
  'dtypes': {'Unnamed: 0': 'int64',
   'country': 'object',
   'description': 'object',
   'designation': 'object',
   'points': 'int64',
   'price': 'float64',
   'province': 'object',
   'region_1': 'object',
   'region_2': 'object',
   'variety': 'object',
   'winery': 'object'},
  'missing_values': {'Unnamed: 0': 0,
   'country': 5,
   'description': 0,
   'designation': 45735,
   'points': 0,
   'price': 13695,

In [9]:
import os

print("Project Structure:")
print(os.listdir())
print("\nAgents Folder:", os.listdir("agents"))
print("\nOutput Folder:", os.listdir("output"))


Project Structure:
['output', '.virtual_documents', 'agents', 'tools']

Agents Folder: ['model_builder.py', '__pycache__', 'feature_analyzer.py', 'dataset_ingestor.py', 'report_generator.py', 'planner_agent.py', 'eda_analyzer.py']

Output Folder: ['report_demo123.md', 'eda', 'models']


In [10]:
from agents.planner_agent import run_pipeline

final_output = run_pipeline("wine-reviews")
final_output


🔹 Run ID: fd04fd0c-9355-4c09-b7be-5234ed0153ad
🔹 Starting pipeline for dataset: wine-reviews
Ingested dataset info: {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv', 'sample_rows': 200, 'sample_cols': 11, 'sample_columns': ['Unnamed: 0', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery']}

📌 Step 1 Completed: Dataset Ingestion
🔍 Starting EDA...
📁 EDA summary saved to: output/eda/fd04fd0c-9355-4c09-b7be-5234ed0153ad_eda.json

📌 Step 2 Completed: EDA Analysis
🔍 Starting Feature Analysis...
📌 Feature analysis completed!

📌 Step 3 Completed: Feature Analysis
Feature Summary (preview): {'numeric_columns': ['Unnamed: 0', 'points', 'price'], 'categorical_columns': ['country', 'description', 'designation']}

🚀 Next steps coming soon: ML Model Builder → Report Generator


{'run_id': 'fd04fd0c-9355-4c09-b7be-5234ed0153ad',
 'dataset_info': {'csv_path': '/kaggle/input/wine-reviews/winemag-data_first150k.csv',
  'sample_rows': 200,
  'sample_cols': 11,
  'sample_columns': ['Unnamed: 0',
   'country',
   'description',
   'designation',
   'points',
   'price',
   'province',
   'region_1',
   'region_2',
   'variety',
   'winery']},
 'eda_summary': {'shape': (150930, 11),
  'columns': ['Unnamed: 0',
   'country',
   'description',
   'designation',
   'points',
   'price',
   'province',
   'region_1',
   'region_2',
   'variety',
   'winery'],
  'dtypes': {'Unnamed: 0': 'int64',
   'country': 'object',
   'description': 'object',
   'designation': 'object',
   'points': 'int64',
   'price': 'float64',
   'province': 'object',
   'region_1': 'object',
   'region_2': 'object',
   'variety': 'object',
   'winery': 'object'},
  'missing_values': {'Unnamed: 0': 0,
   'country': 5,
   'description': 0,
   'designation': 45735,
   'points': 0,
   'price': 13695,